# 💬 4강: AI 피드백 시스템

> 학생 데이터로 피드백 4종(학생/학부모/카톡/4주전략)을 자동 생성합니다.
> **보너스:** 5명 배치(Batch) 피드백!

In [ ]:
!pip install -q google-genai pandas
from google import genai
from google.colab import userdata
import getpass, pandas as pd

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = getpass.getpass('🔑 Gemini API Key: ')

client = genai.Client(api_key=API_KEY)
MODEL = 'gemini-2.5-flash'
print('✅ 준비 완료!')

In [ ]:
# 📊 샘플 학생 데이터
students = pd.DataFrame({
    '이름': ['김민준', '이서연', '박지호', '최수아', '정예준'],
    '학년': ['초4', '초5', '초3', '초6', '초4'],
    '이번점수': [78, 92, 45, 85, 60],
    '지난달': [65, 88, 50, 85, 55],
    '강점': ['어휘', '독해', '듣기', '문법', '쓰기'],
    '약점': ['문법', '쓰기', '어휘', '독해', '문법'],
    '태도': ['적극적', '적극적', '소극적', '보통', '적극적']
})
students['변화'] = students['이번점수'] - students['지난달']
print('📋 학생 데이터:')
students

## 🔬 실습 1: 개별 학생 피드백 4종

In [ ]:
# 첫 번째 학생으로 피드백 4종 생성
s = students.iloc[0]

prompt = f"""
너는 10년 경력 영어학원 원장이자 교육 심리 전문가야.

아래 학생 정보로 피드백 4종을 만들어줘.

학생: {s['이름']} ({s['학년']})
이번 점수: {s['이번점수']}/100 (지난달: {s['지난달']}, 변화: {s['변화']:+d})
강점: {s['강점']} | 약점: {s['약점']} | 태도: {s['태도']}

---

[1] 학생 피드백 (반말, 5줄, 격려형)
🎉 칭찬 → 💪 강점 → 🌱 성장 포인트 → 🎯 이번 주 도전

[2] 학부모 리포트 (존댓말, A4 1장)
📊 성적 요약 → 📈 추이 → 🏆 최고 성과 → 📋 영역별 분석(표) → 🗓️ 다음달 목표 → 💡 가정 가이드

[3] 카톡 메시지 (5줄, 이모지)
인사 → 결과 → 칭찬 → 과제 → CTA

[4] 4주 전략 (표)
| 주차 | 목표 | 활동 | 자료 | 확인 |

"못한다", "부족하다" 금지. 성장형 톤. 점수 하락엔 더 격려.
"""

response = client.models.generate_content(model=MODEL, contents=prompt)
print(f'📋 {s["이름"]} 피드백 4종')
print('='*60)
print(response.text)

## 🔬 실습 2: 5명 배치 피드백 (자동 반복)

In [ ]:
# 🚀 5명 전원 배치 피드백 생성!
import time

all_feedback = {}

for i, s in students.iterrows():
    print(f'\n⏳ {s["이름"]} 피드백 생성 중... ({i+1}/5)')

    prompt = f"""
너는 영어학원 피드백 전문가야. 간결하게 2가지만 만들어줘:

학생: {s['이름']} ({s['학년']})
점수: {s['이번점수']}/100 (변화: {s['변화']:+d})
강점: {s['강점']} | 약점: {s['약점']}

[1] 학생 피드백 (반말, 4줄, 격려)
[2] 카톡 메시지 (5줄, 이모지)

"못한다" 금지. 하락이면 더 격려.
"""
    response = client.models.generate_content(model=MODEL, contents=prompt)
    all_feedback[s['이름']] = response.text
    print(f'  ✅ {s["이름"]} 완료!')
    time.sleep(1)  # API 부하 방지

print('\n🎉 5명 전원 피드백 생성 완료!')
print('='*60)
for name, fb in all_feedback.items():
    print(f'\n👤 {name}')
    print('-'*30)
    print(fb)

In [ ]:
# 💾 전체 저장
output = '\n'.join([f'=== {n} ===\n{fb}' for n, fb in all_feedback.items()])
with open('피드백_배치결과.txt', 'w', encoding='utf-8') as f:
    f.write(output)
from google.colab import files
files.download('피드백_배치결과.txt')
print('✅ 다운로드!')

## 🏆 4강 완료!
👉 [5강: 시험 대비 패키지](https://colab.research.google.com/github/Reasonofmoon/cedu-1/blob/main/notebooks/05_exam_prep.ipynb)